In [1]:
# First cell - Setup
import sys
from pathlib import Path
import nest_asyncio
import asyncio

# Enable async support
nest_asyncio.apply()

# Add the project root directory to Python path
project_root = str(Path().absolute().parent)
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
# Second cell - Fetch data
from providers.nws.observation import fetch_observation_data

# Test fetching observation data
location_id = "NYC"  # Using location ID from settings
data = fetch_observation_data(location_id)

# Print some basic info about the response
print("Raw API Response:")
if 'OBSERVATIONS' in data:
    obs = data['OBSERVATIONS']
    print(f"Found {len(obs.get('date_time', []))} observations")
    print("\nSample data for first observation:")
    print(f"Time: {obs['date_time'][0]}")
    print(f"Temperature: {obs['air_temp_set_1'][0]}°F")
    print(f"Humidity: {obs['relative_humidity_set_1'][0]}%")
    print(f"Wind Speed: {obs['wind_speed_set_1'][0]} mph")
    print(f"Weather: {obs['weather_summary_set_1d'][0]}")
else:
    print("No observations found in data")
print("---")

/Users/leekorff-korn/Code/weather-man/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Raw API Response:
Found 80 observations

Sample data for first observation:
Time: 2024-12-02T22:51:00-0500
Temperature: 33.08°F
Humidity: 53.29%
Wind Speed: 5.75 mph
Weather: clear
---


In [3]:
# Third cell - Clean and load data
from providers.nws.observation import clean_observation_data, load_observation_data

try:
    print("About to start cleaning...")
    cleaned_data = clean_observation_data(data, location_id=location_id)
    print(f"Cleaning complete. Got {len(cleaned_data)} records")
    
    print("Starting database load...")
    records_added = await load_observation_data(cleaned_data)
    print(f"Successfully loaded {records_added} new observation records")
except Exception as e:
    print(f"Error occurred: {str(e)}")
    import traceback
    traceback.print_exc()

About to start cleaning...
Starting observation data cleaning...
Found 80 observations to process

Cleaning complete:
- Processed 80 observations
Cleaning complete. Got 80 records
Starting database load...
2024-12-05 22:09:45,685 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-12-05 22:09:45,686 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-05 22:09:45,823 INFO sqlalchemy.engine.Engine select current_schema()
2024-12-05 22:09:45,824 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-05 22:09:45,969 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-12-05 22:09:45,969 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-05 22:09:46,076 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-05 22:09:46,080 INFO sqlalchemy.engine.Engine SELECT weather_observations.observed_for 
FROM weather_observations 
WHERE weather_observations.location = $1::VARCHAR ORDER BY weather_observations.observed_for DESC 
 LIMIT $2::INTEGER
2024-12-05 22:09:46,081 INFO sqlal